In [1]:
import json
import random
import hashlib
from nacl.signing import SigningKey

# --- Utilities ---
def pack_balance_nonce(balance: int, nonce: int) -> bytes:
    return hashlib.sha256(balance.to_bytes(8, 'little') + nonce.to_bytes(8, 'little')).digest()

class MerkleTree:
    def __init__(self, leaves: list[bytes]):
        n = 1
        while n < len(leaves):
            n <<= 1
        leaves = leaves + [b'\x00' * 32] * (n - len(leaves))
        self.levels = [leaves]
        self._build()

    def _build(self):
        lvl = self.levels[0]
        while len(lvl) > 1:
            nxt = []
            for i in range(0, len(lvl), 2):
                nxt.append(hashlib.sha256(lvl[i] + lvl[i + 1]).digest())
            self.levels.insert(0, nxt)
            lvl = nxt

    @property
    def root(self) -> bytes:
        return self.levels[0][0]

    def get_proof(self, idx: int) -> tuple[list[int], list[bytes]]:
        proof = []
        dirs = []
        for lvl in range(len(self.levels) - 1, 0, -1):
            sib = idx ^ 1
            dirs.append(1 if (idx % 2) == 1 else 0)
            proof.append(self.levels[lvl][sib])
            idx //= 2
        return dirs, proof

# --- Main ---

def main():
    random.seed(42)
    # Load accounts from JSON (expects fields: priv_key, pub_key, balance, nonce)
    with open("../test_accounts/accounts.json", "r") as f:
        accounts = json.load(f)

    # Use pub_key as address
    for acct in accounts:
        acct['addr'] = acct['pub_key']

    # Build initial Merkle tree
    leaves = []
    for acct in accounts:
        addr_bytes = bytes.fromhex(acct['addr'])
        bn_hash = pack_balance_nonce(acct['balance'], acct['nonce'])
        leaves.append(hashlib.sha256(addr_bytes + bn_hash).digest())
    tree = MerkleTree(leaves)
    old_root = tree.root.hex()

    txs = []
    num_txs = 4
    num_acc = len(accounts)

    for _ in range(num_txs):
        # Select a sender with positive balance
        senders = [i for i, a in enumerate(accounts) if a['balance'] > 0]
        if not senders:
            raise RuntimeError("No account with positive balance available")
        sidx = random.choice(senders)
        r_candidates = [i for i in range(num_acc) if i != sidx]
        ridx = random.choice(r_candidates)

        sender = accounts[sidx]
        receiver = accounts[ridx]
        amount = random.randint(1, sender['balance'])
        nonce = sender['nonce']

        # Compute proofs on current tree
        dirs_s, proof_s = tree.get_proof(sidx)
        dirs_r, proof_r = tree.get_proof(ridx)

        # Sign the transaction
        sk = SigningKey(bytes.fromhex(sender['priv_key']))
        recv_bytes = bytes.fromhex(receiver['addr'])
        msg = recv_bytes + amount.to_bytes(8, 'little') + nonce.to_bytes(8, 'little')
        sig = sk.sign(hashlib.sha256(msg).digest()).signature

        tx = {
            'sender_addr': sender['addr'],
            'sender_balance': sender['balance'],
            'sender_nonce': sender['nonce'],
            'sender_proof': [{'dir': d, 'proof': p.hex()} for d, p in zip(dirs_s, proof_s)],
            'receiver_addr': receiver['addr'],
            'amount': amount,
            'tx_nonce': nonce,
            'signature': sig.hex(),
            'pubkey': sender['addr'],
            'recv_balance': receiver['balance'],
            'recv_nonce': receiver['nonce'],
            'recv_proof': [{'dir': d, 'proof': p.hex()} for d, p in zip(dirs_r, proof_r)],
        }
        txs.append(tx)

        # Apply transaction to state and rebuild tree
        sender['balance'] -= amount
        sender['nonce'] += 1
        receiver['balance'] += amount
        # Recompute leaves and tree for next tx
        leaves = []
        for acct in accounts:
            addr_b = bytes.fromhex(acct['addr'])
            bn_h = pack_balance_nonce(acct['balance'], acct['nonce'])
            leaves.append(hashlib.sha256(addr_b + bn_h).digest())
        tree = MerkleTree(leaves)

    new_root = tree.root.hex()

    batch = {
        'old_root': old_root,
        'tx_count': num_txs,
        'txs': txs,
        'new_root': new_root,
    }

    # Write batch to JSON and print
    with open("batch.json", "w") as f:
        json.dump(batch, f, indent=2)
    print(json.dumps(batch, indent=2))

if __name__ == '__main__':
    main()


{
  "old_root": "e9d8e3e9c99d14582b03229b4414a5600faa9ba9f9103017dc7c2048efc0d6c4",
  "tx_count": 4,
  "txs": [
    {
      "sender_addr": "e205c774418c1548169acd9b3c3d4757b09c1848291f4abb9c239e32418a7fb9",
      "sender_balance": 587,
      "sender_nonce": 0,
      "sender_proof": [
        {
          "dir": 1,
          "proof": "9928b6505759e087dbba6394bfa824c223b51270593f7d85a5fb389added88f4"
        },
        {
          "dir": 0,
          "proof": "7b8c42bcb1e2c33b3b2193ac905b9138e045dedc77706e0c2f750d746a75b88d"
        },
        {
          "dir": 0,
          "proof": "d062fa7d49d00d7776aabc2f6e85c87d0b0299b36f3a27f0a8803c6a3b994d29"
        }
      ],
      "receiver_addr": "63badb60ae76c5f3d855130ebbfb53fc0bb731fe893daaf8311b4833cf071b38",
      "amount": 282,
      "tx_nonce": 0,
      "signature": "5c1eb30235c6d3461c85921b325fff277441f51d08c5b96c2ff16540e65f2b66d3aed32f6dac7b81f48ad08b6a38178742d1c7d71c0e75f84cf24fd9aa0a2102",
      "pubkey": "e205c774418c1548169acd9b3